# Teste de Treino Avulso

Objetivo testar se as abstrações criadas permitem o treinamento de um modelo a partir de um dataframe de treino, validação e hiperparâmetros

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lpsds.notebook
import tensorflow as tf
from tensorflow import keras
from tensorflow.image import grayscale_to_rgb
from sklearn.model_selection import train_test_split
import dorothy_sdk.dataset_wrapper as dw
from scripts.operation import ProductionModelOperation

dataset = 'china'
rand_state = 20
device_type = 'gpu'

## Caregando os dados a serem usados no treino

In [ ]:
ds = dw.Dataset(dataset)
df = ds.load_metadata()

trn_df, val_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=rand_state)

trn_df.to_parquet('/bucket/input/trn.parquet')
val_df.to_parquet('/bucket/input/val.parquet')

## Treinando

In [ ]:
hyperparams = {
    'epochs' : 10,
    'batch_size' : 32,
    'learning_rate' : 0.01,
}
with open('/bucket/input/config.json', 'w') as f:
    json.dump(hyperparams, f)

In [ ]:
op = ProductionModelOperation(device_type=device_type)
model = op.train_model(trn_df, val_df, **hyperparams)
model_path = '/bucket/output/models/debugdir'
decision_threshold = 0.6
op.save_model(model, model_path, threshold=decision_threshold);

## Teste de operação

In [ ]:
X_np, y_true = dw.Dataset.load_from_file(df)
X = np.expand_dims(X_np, axis=3)
X = grayscale_to_rgb(tf.constant(X))

In [ ]:
y_proba = model.predict(X)

In [ ]:
op2 = ProductionModelOperation(device_type=device_type)
model2, threshold = op2.load_model(model_path)
y_proba_2, _ = op2.operate_model(model2, X_np, threshold)

In [ ]:
scat_df = pd.DataFrame({'trained' : y_proba.flatten(), 'loaded' : y_proba_2.flatten()})
plt.figure()
sns.scatterplot(data=scat_df, x='trained', y='loaded')
plt.title('Scatterplot entre o Modelo Treinado e de Operação')
plt.xlabel('Modelo treinado (referência)')
plt.ylabel('Modelo de operação')
plt.grid()